In [1]:
import numpy as np, pandas as pd
import seaborn as sns
import RandomCov as rc
import HRP as hrp
import matplotlib.pyplot as mpl
import RCRP as rcrp
%matplotlib inline

In [2]:
# See: https://stackoverflow.com/questions/29351492/how-to-make-a-continuous-alphabetic-list-python-from-a-z-then-from-aa-ab-ac-e
from string import ascii_lowercase
import itertools

def iter_all_strings():
    for size in itertools.count(1):
        for s in itertools.product(ascii_lowercase, repeat=size):
            yield "".join(s)
            
def GetIndexNames(N):
    names = []
    n = 0
    for s in iter_all_strings():
        names.append(s)
        n += 1
        if n >= N:
            return names

In [3]:
# Make a random covariance matrix to run the analysis on
N, k, r, m = 400, 7, 1, 2
np.random.seed(0)
corr   = rc.randomBlockCorr(N, k,random_state=r,minBlockSize=m)
indexNames = GetIndexNames(N)
corr.index, corr.columns = indexNames, indexNames
sigma  = pd.Series( 1. + np.random.rand(N) * 10., index = corr.index )
#sharpe = pd.Series( np.random.lognormal(mean=-1./200, sigma = 0.1, size=N), index = corr.index )
sharpe = pd.Series( np.random.gamma(1., size=N), index = corr.index )
# Make a random set of returns
# Set mu near sigma so that sharpe ratio is near 1 everywhere
mu     = sigma * sharpe
cov    = corr * pd.DataFrame( np.outer(sigma,sigma), index=corr.index, columns=corr.columns )

In [4]:
def stats(w, cov, mu):
    m = np.dot( w , mu )
    s = np.dot( w.T, np.dot( cov, w ) )
    sharpe = m / s
    w2 = w.abs()
    PP = 'mu = ' + str( np.round(m,3) )
    PP += ' sigma = ' + str( np.round( s, 3 ) )
    PP += ' sharpe = ' + str( np.round( sharpe, 3 ) )
    PP += ' sum of weights error = ' + str( np.round( np.abs(1-w.sum()) , 12 ) )
    PP += ' max |w| = ' + str( np.round( w.abs().max() , 3 ) )
    #print 'mu =', np.round(m, 'sigma =', s, 'sharpe =', sharpe , 'sum of weights error =', np.abs(1-w.sum()), 'max |w| =', w.abs().max()
    print PP
    print 'number of large w:', len( w2[w2>w2.mean()+w2.std()*4] )
    return
#------------------------------------------------------------------------------
def Get_HRP_sortIx(corr):
    import scipy.cluster.hierarchy as sch
    dist=hrp.correlDist(corr)
    link=sch.linkage(dist,'single')
    sortIx=hrp.getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist() # recover labels
    return sortIx

In [5]:
print 'min var'
w = pd.Series( np.dot( np.linalg.inv(cov), np.ones(N) ), index = cov.index )
w /= w.sum()
stats( w, cov, mu )
print ''

print 'max sharpe'
w = pd.Series( np.dot( np.linalg.inv(cov), mu ), index = cov.index )
w /= w.sum()
stats( w, cov, mu )
print ''

print 'IVP'
w = pd.Series( hrp.getIVP(cov), index = cov.index )
stats( w, cov, mu )
print ''

print 'IVPNew, extended terms'
w = pd.Series( rcrp.getIVPNew(cov, use_extended_terms=True, a=None), index = cov.index )
stats( w, cov, mu )
print ''

print 'IVPNew, max sharpe'
w = pd.Series( rcrp.getIVPNew(cov, use_extended_terms=False, a=mu), index = cov.index )
stats( w, cov, mu )
print ''

print 'IVPNew, max sharpe, extended terms'
w = pd.Series( rcrp.getIVPNew(cov, use_extended_terms=True, a=mu), index = cov.index )
stats( w, cov, mu )
print ''

sortIx = Get_HRP_sortIx(corr)
print 'HRP'
w = hrp.getRecBipart(cov, sortIx)
stats( w, cov, mu )
print ''

print 'HRPNew'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=False, use_extended_terms2=False, returns=None)
stats( w, cov, mu )
print ''

print 'HRPNew, extended terms1'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=True, use_extended_terms2=False, returns=None)
stats( w, cov, mu )
print ''

print 'HRPNew, extended terms1&2'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=True, use_extended_terms2=True, returns=None)
stats( w, cov, mu )
print ''

print 'HRPNew, max sharpe'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=False, use_extended_terms2=False, returns=mu)
stats( w, cov, mu )
print ''

print 'HRPNew, max sharpe, extended terms1'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=True, use_extended_terms2=False, returns=mu)
stats( w, cov, mu )
print ''

print 'HRPNew, max sharpe, extended terms1&2'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=True, use_extended_terms2=True, returns=mu)
stats( w, cov, mu )
print ''

print 'RCRP'
w = rcrp.RCRP( cov )
stats( w, cov, mu )
print ''

print 'RCRP, no extended terms'
w = rcrp.RCRP( cov, use_extended_terms=False )
stats( w, cov, mu )
print ''

print 'RCRP, max sharpe'
w = rcrp.RCRP( cov, a=mu )
stats( w, cov, mu )
print ''

print 'RCRP, no extended terms, max sharpe'
w = rcrp.RCRP( cov, use_extended_terms=False, a=mu )
stats( w, cov, mu )
print ''

min var
mu = -0.17 sigma = 0.029 sharpe = -5.911 sum of weights error = 0.0 max |w| = 0.054
number of large w: 10

max sharpe
mu = -216.283 sigma = 36.59 sharpe = -5.911 sum of weights error = 0.0 max |w| = 1.001
number of large w: 3

IVP
mu = 2.622 sigma = 2.557 sharpe = 1.026 sum of weights error = 0.0 max |w| = 0.025
number of large w: 6

IVPNew, extended terms
mu = -0.174 sigma = 0.048 sharpe = -3.596 sum of weights error = 0.0 max |w| = 0.05
number of large w: 12

IVPNew, max sharpe
mu = 9.477 sigma = 6.665 sharpe = 1.422 sum of weights error = 0.0 max |w| = 0.029
number of large w: 5

IVPNew, max sharpe, extended terms
mu = -210.537 sigma = 59.72 sharpe = -3.525 sum of weights error = 0.0 max |w| = 0.859
number of large w: 3



/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:19: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix


HRP
mu = 5.002 sigma = 10.608 sharpe = 0.472 sum of weights error = 0.0 max |w| = 0.04
number of large w: 7

HRPNew
mu = 5.002 sigma = 10.608 sharpe = 0.472 sum of weights error = 0.0 max |w| = 0.04
number of large w: 7

HRPNew, extended terms1
mu = 25.334 sigma = 179.799 sharpe = 0.141 sum of weights error = 0.0 max |w| = 2.24
number of large w: 4

HRPNew, extended terms1&2
mu = 5.576 sigma = 9.104 sharpe = 0.612 sum of weights error = 0.0 max |w| = 0.069
number of large w: 9

HRPNew, max sharpe
mu = 5.475 sigma = 11.808 sharpe = 0.464 sum of weights error = 0.0 max |w| = 0.043
number of large w: 2

HRPNew, max sharpe, extended terms1
mu = -195701.031 sigma = 33019096239.596 sharpe = -0.0 sum of weights error = 2e-12 max |w| = 30455.757
number of large w: 4

HRPNew, max sharpe, extended terms1&2
mu = 391.165 sigma = 25997.283 sharpe = 0.015 sum of weights error = 0.0 max |w| = 11.011
number of large w: 6

RCRP


clusterCorr.py:50: RuntimeWarning: invalid value encountered in double_scalars
  clusterTstats={i:np.mean(silh[ clstrs[i]])/np.std(silh[clstrs[i]]) for i in clstrs.keys()}


mu = 0.147 sigma = 0.062 sharpe = 2.365 sum of weights error = 0.0 max |w| = 0.073
number of large w: 8

RCRP, no extended terms
mu = 2.625 sigma = 2.408 sharpe = 1.09 sum of weights error = 0.0 max |w| = 0.096
number of large w: 5

RCRP, max sharpe
mu = 100.265 sigma = 24.884 sharpe = 4.029 sum of weights error = 0.0 max |w| = 0.783
number of large w: 3

RCRP, no extended terms, max sharpe
mu = 8.935 sigma = 6.651 sharpe = 1.343 sum of weights error = 0.0 max |w| = 0.089
number of large w: 4

